# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [28]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [29]:
df = pd.read_csv('previsao_de_renda.csv')

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   index                  15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12466 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  mau                    15000 non-null  bool   
 15  re

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [31]:
df.drop('Unnamed: 0',axis=1,inplace=True)
df['sexo'] = df['sexo'].map({'F':0,'M':1})
df['posse_de_veiculo'] = df['posse_de_veiculo'].astype(int)
df['posse_de_imovel'] = df['posse_de_imovel'].astype(int)
df['mau'] = df['mau'].astype(int)
df['data_ref'] = pd.to_datetime(df['data_ref']) 
df.dropna(inplace=True)
df.reset_index(inplace= True)
df_clear = pd.get_dummies(df,columns=['tipo_residencia','estado_civil','educacao','tipo_renda'],drop_first=True)
df_clear['log_renda'] = np.log(df_clear['renda'])
df_clear.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12466 entries, 0 to 12465
Data columns (total 30 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   level_0                        12466 non-null  int64         
 1   data_ref                       12466 non-null  datetime64[ns]
 2   index                          12466 non-null  int64         
 3   sexo                           12466 non-null  int64         
 4   posse_de_veiculo               12466 non-null  int32         
 5   posse_de_imovel                12466 non-null  int32         
 6   qtd_filhos                     12466 non-null  int64         
 7   idade                          12466 non-null  int64         
 8   tempo_emprego                  12466 non-null  float64       
 9   qt_pessoas_residencia          12466 non-null  float64       
 10  mau                            12466 non-null  int32         
 11  renda          

In [32]:
X = df_clear.drop(columns=['data_ref', 'index', 'renda','log_renda'])
y = df_clear['log_renda']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

---

In [33]:
from sklearn.linear_model import Ridge,Lasso
from sklearn.metrics import r2_score

alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
ridge_scores = {}

for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train, y_train)
    y_pred = ridge.predict(X_test)
    ridge_scores[alpha] = r2_score(y_test, y_pred)

best_ridge_alpha = max(ridge_scores, key=ridge_scores.get)
best_ridge_score = ridge_scores[best_ridge_alpha]
best_ridge_model = Ridge(alpha=best_ridge_alpha).fit(X_train, y_train)
print(f"Best Ridge alpha: {best_ridge_alpha}, R^2: {best_ridge_score}")
print(f"Ridge coefficients: {best_ridge_model.coef_}")


Best Ridge alpha: 0.1, R^2: 0.23387132527927568
Ridge coefficients: [-1.50065200e-06  4.84023315e-01 -3.08822164e-03  1.05938045e-01
  1.19014245e-01  3.94561474e-03  4.92750746e-02 -1.06313005e-01
 -2.25980652e-03  7.62739733e-03 -7.27561656e-02 -3.03729621e-02
 -5.11636386e-02 -3.96563411e-02 -1.52958797e-01 -9.63821076e-02
 -1.71127020e-02 -1.28250667e-01  4.54617807e-01 -8.78491961e-03
  8.51644716e-02 -2.61585789e-02 -1.84249000e-02  1.67022686e-01
  3.77972226e-01  8.54747171e-02]


---

In [34]:
lasso_scores = {}

for alpha in alphas:
    lasso = Lasso(alpha=alpha, max_iter=10000)  # Aumentar o número de iterações para garantir a convergência
    lasso.fit(X_train, y_train)
    y_pred = lasso.predict(X_test)
    lasso_scores[alpha] = r2_score(y_test, y_pred)

best_lasso_alpha = max(lasso_scores, key=lasso_scores.get)
best_lasso_score = lasso_scores[best_lasso_alpha]
best_lasso_model = Lasso(alpha=best_lasso_alpha, max_iter=10000).fit(X_train, y_train)
print(f"Best LASSO alpha: {best_lasso_alpha}, R^2: {best_lasso_score}")
print(f"LASSO coefficients: {best_lasso_model.coef_}")



C:\Users\User\AppData\Local\Temp\ipykernel_13716\2932569161.py:5: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso.fit(X_train, y_train)
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.281e+03, tolerance: 6.032e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\Users

Best LASSO alpha: 0, R^2: 0.2338600853220969
LASSO coefficients: [-1.50040080e-06  4.84057686e-01 -3.10887786e-03  1.05935317e-01
  1.24031774e-01  3.94599598e-03  4.92749410e-02 -1.11326716e-01
 -2.29804674e-03  7.48528529e-03 -7.29079389e-02 -3.05214498e-02
 -5.13831764e-02 -3.97947921e-02 -1.57997013e-01 -1.01386089e-01
 -1.71192680e-02 -1.33310854e-01  4.58172483e-01 -8.07757348e-03
  8.58732058e-02 -2.54580525e-02 -1.90338818e-02  1.67026481e-01
  3.83410853e-01  8.54999400e-02]


C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.281e+03, tolerance: 6.032e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


---

In [35]:
def forward_stepwise_selection(X_train, y_train, X_test, y_test):
    remaining_features = list(X_train.columns)
    current_features = []
    best_r2 = 0.0

    while remaining_features:
        r2_with_new_feature = []

        for feature in remaining_features:
            features_to_use = current_features + [feature]
            X_train_sm = sm.add_constant(X_train[features_to_use])
            X_test_sm = sm.add_constant(X_test[features_to_use])
            model = sm.OLS(y_train, X_train_sm).fit()
            predictions = model.predict(X_test_sm)
            r2 = r2_score(y_test, predictions)
            r2_with_new_feature.append((r2, feature))

        r2_with_new_feature.sort(reverse=True)
        best_r2_new, best_new_feature = r2_with_new_feature[0]

        if best_r2_new > best_r2:
            remaining_features.remove(best_new_feature)
            current_features.append(best_new_feature)
            best_r2 = best_r2_new
        else:
            break

    return current_features, best_r2

selected_features, best_r2 = forward_stepwise_selection(X_train, y_train, X_test, y_test)
print("Melhores recursos selecionados:", selected_features)
print("Melhor R^2:", best_r2)


X_train_selected = sm.add_constant(X_train[selected_features])
X_test_selected = sm.add_constant(X_test[selected_features])
stepwise_model = sm.OLS(y_train, X_train_selected).fit()
y_pred = stepwise_model.predict(X_test_selected)
stepwise_score = r2_score(y_test, y_pred)
print(f"Stepwise model R^2: {stepwise_score}")
print(f"Stepwise coefficients: {stepwise_model.params}")


Melhores recursos selecionados: ['tempo_emprego', 'sexo', 'tipo_renda_Empresário', 'idade', 'educacao_Superior completo', 'tipo_renda_Servidor público', 'tipo_residencia_Casa', 'educacao_Pós graduação', 'qt_pessoas_residencia', 'estado_civil_Solteiro', 'tipo_residencia_Comunitário', 'qtd_filhos', 'estado_civil_União', 'level_0', 'tipo_renda_Pensionista', 'educacao_Secundário', 'mau']
Melhor R^2: 0.23685615438531893
Stepwise model R^2: 0.23685615438531893
Stepwise coefficients: const                          7.163726
tempo_emprego                  0.049057
sexo                           0.482741
tipo_renda_Empresário          0.167135
idade                          0.004416
educacao_Superior completo     0.116355
tipo_renda_Servidor público    0.085422
tipo_residencia_Casa           0.090099
educacao_Pós graduação         0.518324
qt_pessoas_residencia          0.036590
estado_civil_Solteiro          0.049774
tipo_residencia_Comunitário    0.039530
qtd_filhos                    -0.02049

---

In [36]:
# Ridge com melhor alpha
best_ridge_model = Ridge(alpha=best_ridge_alpha).fit(X_train, y_train)
print(f"Ridge coefficients: {best_ridge_model.coef_}")

# LASSO com melhor alpha
best_lasso_model = Lasso(alpha=best_lasso_alpha).fit(X_train, y_train)
print(f"LASSO coefficients: {best_lasso_model.coef_}")

# Stepwise model
print(f"Stepwise coefficients: {stepwise_model.params}")


Ridge coefficients: [-1.50065200e-06  4.84023315e-01 -3.08822164e-03  1.05938045e-01
  1.19014245e-01  3.94561474e-03  4.92750746e-02 -1.06313005e-01
 -2.25980652e-03  7.62739733e-03 -7.27561656e-02 -3.03729621e-02
 -5.11636386e-02 -3.96563411e-02 -1.52958797e-01 -9.63821076e-02
 -1.71127020e-02 -1.28250667e-01  4.54617807e-01 -8.78491961e-03
  8.51644716e-02 -2.61585789e-02 -1.84249000e-02  1.67022686e-01
  3.77972226e-01  8.54747171e-02]


C:\Users\User\AppData\Local\Temp\ipykernel_13716\3585891120.py:6: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  best_lasso_model = Lasso(alpha=best_lasso_alpha).fit(X_train, y_train)
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(


LASSO coefficients: [-1.50681107e-06  4.84091213e-01 -3.12287397e-03  1.05946785e-01
  9.08848541e-02  3.94249470e-03  4.92788896e-02 -7.82537957e-02
 -3.22236041e-03  7.41750576e-03 -7.29616225e-02 -3.05659594e-02
 -5.14066673e-02 -3.98194988e-02 -1.24916257e-01 -6.85839382e-02
 -1.70404121e-02 -1.00214415e-01  4.58239223e-01 -8.06704706e-03
  8.58870255e-02 -2.54072506e-02 -1.90894515e-02  1.67083275e-01
  3.84411436e-01  8.54863884e-02]
Stepwise coefficients: const                          7.163726
tempo_emprego                  0.049057
sexo                           0.482741
tipo_renda_Empresário          0.167135
idade                          0.004416
educacao_Superior completo     0.116355
tipo_renda_Servidor público    0.085422
tipo_residencia_Casa           0.090099
educacao_Pós graduação         0.518324
qt_pessoas_residencia          0.036590
estado_civil_Solteiro          0.049774
tipo_residencia_Comunitário    0.039530
qtd_filhos                    -0.020499
estado_civil_

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.281e+03, tolerance: 6.032e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


In [42]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

# Definir os parâmetros para a busca em grade
param_grid = {
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'auto', 'sqrt', 'log2']
}

# Ajustar a árvore de regressão usando GridSearchCV
tree = DecisionTreeRegressor(random_state=42)
grid_search = GridSearchCV(estimator=tree, param_grid=param_grid, cv=5, n_jobs=-1, scoring='r2')
grid_search.fit(X_train, y_train)

# Prever e calcular o R^2 com os melhores parâmetros encontrados
best_tree = grid_search.best_estimator_
y_pred = best_tree.predict(X_test)
best_tree_r2 = r2_score(y_test, y_pred)

print(f"Best Decision Tree parameters: {grid_search.best_params_}")
print(f"Best Decision Tree R^2: {best_tree_r2}")



Best Decision Tree parameters: {'max_depth': 5, 'max_features': None, 'min_samples_leaf': 2, 'min_samples_split': 10}
Best Decision Tree R^2: 0.2093347228505279
